# `Word2Vec`

In [96]:
import nltk
import numpy as np
import tensorflow as tf

In [112]:
corpus_raw = 'He is the king. The king is royal. She is the royal  queen '
corpus_raw = corpus_raw.lower()

In [113]:
# Tokenization
words = nltk.word_tokenize(corpus_raw)
raw_sentences = nltk.sent_tokenize(corpus_raw)

In [114]:
# Word2ID and ID2Word
id2word = {i: w for i, w in enumerate(words)}
word2id = {w: i for i, w in enumerate(words)}

In [115]:
print(id2word, '\n', word2id)

{0: 'he', 1: 'is', 2: 'the', 3: 'king', 4: '.', 5: 'the', 6: 'king', 7: 'is', 8: 'royal', 9: '.', 10: 'she', 11: 'is', 12: 'the', 13: 'royal', 14: 'queen'} 
 {'he': 0, 'is': 11, 'the': 12, 'king': 6, '.': 9, 'royal': 13, 'she': 10, 'queen': 14}


In [116]:
word2id['queen']

14

In [117]:
id2word[14]

'queen'

In [120]:
# List of sentence as list of words
sentences = [nltk.word_tokenize(sent) for sent in raw_sentences]
print(sentences[:5])

[['he', 'is', 'the', 'king', '.'], ['the', 'king', 'is', 'royal', '.'], ['she', 'is', 'the', 'royal', 'queen']]


### Generate training data

In [124]:
data = []
WINDOW_SIZE = 2  # ['WINDOW', 'WINDOW', 'TARGET', 'WINDOW', 'WINDOW']
for sent in sentences:
    for word_idx, word in enumerate(sent):
        start = max(word_idx - WINDOW_SIZE, 0)
        end = min(word_idx + WINDOW_SIZE, len(sent)+1)
        for win in sent[start:end]:
            if win != word:
                data.append([word, win])

In [134]:
print(data[:5])

[['he', 'is'], ['is', 'he'], ['is', 'the'], ['the', 'he'], ['the', 'is']]
